In [1]:
ANIMAL = "chimp"

In [2]:
# Parameters
ANIMAL = "degus"


In [3]:
import pathlib
import keras
import tensorflow as tf
import plotly.graph_objects as go
import contextlib
import io

from petface.const import IMG_HEIGHT, IMG_WIDTH, ROOT_DIR
from petface.models.face_identification.res_net.model import get_reidentification_model

with contextlib.redirect_stdout(io.StringIO()):
    IMG_DIMENSIONS = (IMG_HEIGHT, IMG_WIDTH)

    root_dir = pathlib.Path(ROOT_DIR)
    test_dir = root_dir / "dataset" / "out" / "images" / "test" / ANIMAL

    test_ds = keras.utils.image_dataset_from_directory(
        directory=test_dir,
        image_size=IMG_DIMENSIONS,
        batch_size=32
    )

    eval_ds = test_ds.prefetch(tf.data.AUTOTUNE)  # type: ignore

2025-11-29 19:17:00.674546: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-29 19:17:00.690033: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
with contextlib.redirect_stdout(io.StringIO()):
    softmax_cross_entropy_model = get_reidentification_model(
        animal=ANIMAL, loss_type="cross_entropy", classification_type="softmax"
    )
    softmax_similarity_model = get_reidentification_model(
        animal=ANIMAL, loss_type="cross_entropy", classification_type="cosine_similarity"
    )
    arcface_cross_entropy_model = get_reidentification_model(
        animal=ANIMAL, loss_type="arcface", classification_type="softmax"
    )
    arcface_similarity_model = get_reidentification_model(
        animal=ANIMAL, loss_type="arcface", classification_type="cosine_similarity"
    )

    softmax_cross_entropy_model.load_from_checkpoint()
    softmax_similarity_model.load_from_checkpoint()
    arcface_cross_entropy_model.load_from_checkpoint()
    arcface_similarity_model.load_from_checkpoint()

/home/mtofano/micromamba/envs/petface/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 84 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


/home/mtofano/micromamba/envs/petface/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 80 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


/home/mtofano/micromamba/envs/petface/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
softmax_cross_entropy_model.display_training_history()

In [6]:
softmax_similarity_model.display_training_history()

In [7]:
arcface_cross_entropy_model.display_training_history()

In [8]:
arcface_similarity_model.display_training_history()

In [9]:
with contextlib.redirect_stdout(io.StringIO()):
    softmax_cross_entropy_eval_results = softmax_cross_entropy_model.evaluate(eval_ds)
    softmax_similarity_eval_results = softmax_similarity_model.evaluate(eval_ds)
    arcface_cross_entropy_eval_results = arcface_cross_entropy_model.evaluate(eval_ds)
    arcface_similarity_eval_results = arcface_similarity_model.evaluate(eval_ds)

2025-11-29 19:17:14.250600: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-29 19:17:14.323192: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


2025-11-29 19:18:07.296117: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
eval_table = go.Figure(
    data=[
        go.Table(
            header={"values": ["Loss", "Classifier", "Top 1 Accuracy", "Top 5 Accuracy", "Top 50 Accuracy"]},
            cells={
                "values": [
                    ["Cross Entropy", "Cross Entropy", "Arcface", "Arcface"],
                    ["Softmax", "Cosine Similarity", "Softmax", "Cosine Similarity"],
                    [
                        softmax_cross_entropy_eval_results["top_1_accuracy"],
                        softmax_similarity_eval_results["top_1_accuracy"],
                        arcface_cross_entropy_eval_results["top_1_accuracy"],
                        arcface_similarity_eval_results["top_1_accuracy"]
                    ],
                    [
                        softmax_cross_entropy_eval_results["top_5_accuracy"],
                        softmax_similarity_eval_results["top_5_accuracy"],
                        arcface_cross_entropy_eval_results["top_5_accuracy"],
                        arcface_similarity_eval_results["top_5_accuracy"]
                    ],
                    [
                        softmax_cross_entropy_eval_results["top_50_accuracy"],
                        softmax_similarity_eval_results["top_50_accuracy"],
                        arcface_cross_entropy_eval_results["top_50_accuracy"],
                        arcface_similarity_eval_results["top_50_accuracy"]
                    ],
                ]
            }
        )
    ]
)

eval_table.show()